In [2]:
import json
import pandas as pd
from tqdm import tqdm
from pathlib import Path 
from ast import literal_eval

## Annotations

Collected tweets are linked to DBpedia and Wikidata with `spacy` and two libraries built on top of `spacy`:
* [DBpedia Spotlight for SpaCy](https://spacy.io/universe/project/spacy-dbpedia-spotlight)
* [spaCyOpenTapioca](https://spacy.io/universe/project/spacyopentapioca)

In [39]:
db_tweets = pd.read_csv('../data/db_annotated_tweets.csv', index_col=False)
db_tweets = db_tweets.fillna('')

# db_user_tweets = pd.read_csv('../data/db_annotated_user_tweets.csv', index_col=False)
# db_user_tweets = db_user_tweets.fillna('')

# db_tweets = pd.concat([db_tweets, db_user_tweets])

C:\Users\gokce\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (0,1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
wd_tweets = pd.read_csv('../data/wd_annotated_tweets.csv', index_col=False)
wd_tweets = wd_tweets.fillna('')

In [38]:
db_user_tweets

,Unnamed: 0,index,Unnamed: 0.1,Timestamp,TweetID,TweetText,Entities,UserID,UserName,retweet
0,16,16,16,Thu Nov 04 20:45:01 +0000 2021,1456361867694252041,"Girl, 15, is suspended after reporting sexual ...","{'hashtags': [], 'symbols': [], 'user_mentions...",246942558.0,1234renodog,
1,17,17,17,Wed Nov 03 09:37:39 +0000 2021,1455831531285528576,. https://t.co/f8HzedUAxn,"{'hashtags': [], 'symbols': [], 'user_mentions...",246942558.0,1234renodog,
2,18,18,18,Sat Nov 13 16:10:40 +0000 2021,1459554314553376771,. https://t.co/P6Vk2UewsO,"{'hashtags': [], 'symbols': [], 'user_mentions...",246942558.0,1234renodog,
3,19,19,19,Fri Nov 26 20:00:18 +0000 2021,1464323144592801799,. https://t.co/wGlhJuqDEj,"{'hashtags': [], 'symbols': [], 'user_mentions...",246942558.0,1234renodog,
4,20,20,20,Sun Nov 07 16:41:02 +0000 2021,1457387629696020481,@RealJamesWoods waiting for more ballots to sh...,"{'hashtags': [], 'symbols': [], 'user_mentions...",246942558.0,1234renodog,
...,...,...,...,...,...,...,...,...,...,...
42751,42765,42765,42765,Sat Dec 11 09:23:28 +0000 2021,1469598699596783620,@JeanLuiseFinch Hangi diziydi bu acaba? İzledi...,"{'hashtags': [], 'symbols': [], 'user_mentions...",757556294110838784.0,gokceuludogan_,
42752,42766,42766,42766,Sun Oct 31 07:40:55 +0000 2021,1454714989449785347,"@AydKub Hadi öğrendin, niye paylaşıp bizim de ...","{'hashtags': [], 'symbols': [], 'user_mentions...",757556294110838784.0,gokceuludogan_,
42753,42767,42767,42767,Wed Oct 27 18:44:28 +0000 2021,1453432427309936641,@tunckuyucu Akademik ders müfredatı.. Sarlatan...,"{'hashtags': [], 'symbols': [], 'user_mentions...",757556294110838784.0,gokceuludogan_,
42754,42768,42768,42768,Sat Dec 11 09:29:09 +0000 2021,1469600131754381312,@JeanLuiseFinch Tamamdır teşekkürler :),"{'hashtags': [], 'symbols': [], 'user_mentions...",757556294110838784.0,gokceuludogan_,


In [ ]:
db_tweets['db_ents_list'] = db_tweets['db_ents'].apply(lambda x: [item.split(',') for item in x.replace('"', '').split(';')])
wd_tweets['wd_ents_list'] = wd_tweets['wd_ents'].apply(lambda x: [item.split(',') for item in x.replace('"', '').split(';')])

## Entity and Entity Type Counts

#### DBpedia

In [ ]:
db_ent_token_counts = db_tweets['db_ents_list'].apply(lambda x: [i[0] for i in x]).explode().value_counts().reset_index()
db_ent_token_counts

In [31]:
db_type_counts = db_tweets['db_ents_list'].apply(lambda x: [i[1] if len(i) >= 2 else '' for i in x]).explode().value_counts().reset_index()
db_type_counts

,index,db_ents_list
0,DBPEDIA_ENT,386765
1,ORG,50893
2,,21677
3,PERSON,18464
4,DATE,5577
...,...,...
121,McMichaels,1
122,IL,1
123,minutes and seconds,1
124,DRUG DEALERS,1


In [32]:
db_ent_counts = db_tweets['db_ents_list'].apply(lambda x: [i[2] if len(i) >= 3 else '' for i in x]).explode().value_counts().reset_index()
db_ent_counts

,index,db_ents_list
0,http://dbpedia.org/resource/Kyle_Broflovski,116890
1,,107365
2,http://dbpedia.org/resource/White_supremacy,8664
3,http://dbpedia.org/resource/Wisconsin,8078
4,http://dbpedia.org/resource/VAX,6656
...,...,...
14329,http://dbpedia.org/resource/Madonna_(entertainer),1
14330,http://dbpedia.org/resource/Anthony_Cumia,1
14331,http://dbpedia.org/resource/Punxsutawney_Phil,1
14332,http://dbpedia.org/resource/Limerick_Institute...,1


#### Wikidata

In [33]:
wd_tweets['wd_ents_list'].apply(lambda x: [(i[0], i[2]) if len(i) >= 3 else '' for i in x]).explode().value_counts().reset_index()[:50]

,index,wd_ents_list
0,"(Kyle Rittenhouse, Q98663135)",68147
1,"(Rittenhouse, )",39861
2,,25173
3,"(Kyle Rittenhouse, )",15439
4,"(video, Q9022)",5687
5,"(A, Q48)",5479
6,"(can, Q16)",5119
7,"(Kyle Rittenhouses, )",5074
8,"(American, )",4613
9,"(Kyle Rittenhouse's, Q98663135)",4598


In [21]:
wd_ent_token_counts = wd_tweets['wd_ents_list'].apply(lambda x: [i[0] for i in x]).explode().value_counts().reset_index()
wd_ent_token_counts

,index,wd_ents_list
0,Kyle Rittenhouse,83588
1,Rittenhouse,41472
2,,24210
3,video,5687
4,A,5479
...,...,...
24778,Gisselle Maxwell,1
24779,Q Folks,1
24780,Latter Day Saint,1
24781,mecican,1


In [22]:
wd_ent_type_counts = wd_tweets['wd_ents_list'].apply(lambda x: [i[1] if len(i) >= 2 else '' for i in x]).explode().value_counts().reset_index()
wd_ent_type_counts[:20]

,index,wd_ents_list
0,PERSON,187755
1,ORG,99627
2,LOC,77971
3,,24354
4,DATE,12271
5,GPE,9626
6,NORP,7083
7,ORGLOC,2725
8,ORDINAL,2473
9,WORK_OF_ART,1509


In [8]:
# Mentions
# wd_tweets['MentionedUser'] = wd_tweets['mentions'].apply(lambda x: [i.split(',')[0] for i in x.split(';')])
# mentions = wd_tweets.explode('MentionedUser')[['UserName', 'MentionedUser']]

# mentions['Source'] = mentions['UserName']
# mentions['Target'] = mentions['MentionedUser']

# mentions[['Source', 'Target']].to_csv('../data/mention.csv')

In [10]:
db_entities = db_tweets.explode('db_ents_list')[['UserName', 'TweetText', 'db_ents_list']]

In [11]:
db_entities['ent_type'] = db_entities['db_ents_list'].apply(lambda x: x[1] if len(x) > 1 else '')
db_entities

,UserName,TweetText,db_ents_list,ent_type
0,@JoyceGarbaciak,Judge sets final evidence rules for before Kyl...,"[Kyle, DBPEDIA_ENT, http://dbpedia.org/resourc...",DBPEDIA_ENT
0,@JoyceGarbaciak,Judge sets final evidence rules for before Kyl...,"[Kyle, DBPEDIA_ENT, http://dbpedia.org/resourc...",DBPEDIA_ENT
0,@JoyceGarbaciak,Judge sets final evidence rules for before Kyl...,"[Kenosha County, DBPEDIA_ENT, http://dbpedia.o...",DBPEDIA_ENT
0,@JoyceGarbaciak,Judge sets final evidence rules for before Kyl...,"[Kyle, DBPEDIA_ENT, http://dbpedia.org/resourc...",DBPEDIA_ENT
1,@atdavidhoffman,"Guns don’t kill people,\nKyle Rittenhouse kill...","[Kyle, DBPEDIA_ENT, http://dbpedia.org/resourc...",DBPEDIA_ENT
...,...,...,...,...
231709,HunderthMonkey,"@CopperBowman @BOOMER84901196 @MiaCathell Ok, ...","[Covington, DBPEDIA_ENT, http://dbpedia.org/re...",DBPEDIA_ENT
231709,HunderthMonkey,"@CopperBowman @BOOMER84901196 @MiaCathell Ok, ...","[Sandman, DBPEDIA_ENT, http://dbpedia.org/reso...",DBPEDIA_ENT
231709,HunderthMonkey,"@CopperBowman @BOOMER84901196 @MiaCathell Ok, ...","[Native American, DBPEDIA_ENT, http://dbpedia....",DBPEDIA_ENT
231709,HunderthMonkey,"@CopperBowman @BOOMER84901196 @MiaCathell Ok, ...","[Vietnam, DBPEDIA_ENT, http://dbpedia.org/reso...",DBPEDIA_ENT


In [12]:
db_entities['ent_type'].value_counts().reset_index()[:10]

,index,ent_type
0,DBPEDIA_ENT,386765
1,ORG,50893
2,,21677
3,PERSON,18464
4,DATE,5577
5,GPE,4777
6,ORDINAL,1744
7,NORP,1501
8,TIME,912
9,WORK_OF_ART,673


In [13]:
db_ents_with_id = db_entities[db_entities['ent_type'] == 'DBPEDIA_ENT']
db_ents_with_id['wikidata'] = db_ents_with_id.apply(lambda x: [item for item in x['db_ents_list'][3:] if 'Wikidata' in item], axis=1)
db_ents_with_id['dbpedia'] = db_ents_with_id.apply(lambda x: [item for item in x['db_ents_list'][3:] if 'DBpedia' in item], axis=1)

C:\Users\gokce\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\gokce\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
db_ents_with_id_dbp = db_ents_with_id.explode('dbpedia')
db_ents_with_id['dbpedia']

0               [DBpedia:Agent, DBpedia:FictionalCharacter]
0               [DBpedia:Agent, DBpedia:FictionalCharacter]
0         [DBpedia:Region, DBpedia:PopulatedPlace, DBped...
0               [DBpedia:Agent, DBpedia:FictionalCharacter]
1               [DBpedia:Agent, DBpedia:FictionalCharacter]
                                ...                        
231709                                                   []
231709    [DBpedia:Person, DBpedia:Athlete, DBpedia:Agen...
231709                                                   []
231709                                                   []
231709    [DBpedia:PopulatedPlace, DBpedia:Place, DBpedi...
Name: dbpedia, Length: 386765, dtype: object

In [18]:
dbr_counts = db_ents_with_id[db_ents_with_id['dbpedia'].apply(len) == 0]['db_ents_list'].apply(lambda x: x[2] if len(x) > 2 else '').value_counts().reset_index()
dbr_counts

,index,db_ents_list
0,http://dbpedia.org/resource/White_supremacy,8664
1,http://dbpedia.org/resource/VAX,6656
2,http://dbpedia.org/resource/Hurricane_Floyd,5319
3,http://dbpedia.org/resource/Self-defense,3558
4,http://dbpedia.org/resource/Kenosha_Comets,3348
...,...,...
6910,http://dbpedia.org/resource/Acro_dance,1
6911,http://dbpedia.org/resource/Oberheim_DMX,1
6912,http://dbpedia.org/resource/Dragon_Ball,1
6913,http://dbpedia.org/resource/Working_animal,1


In [20]:
dbr_counts = dbr_counts.rename(columns={'index':'uri','db_ents_list':'count'})
dbr_counts.to_csv('../data/dbpedia_resources.csv', index=False)

### Entity Cooccurrences

In [28]:
from itertools import product
entity_counts = db_ents_with_id['dbpedia'].apply(lambda x: list(product(x, x))).explode().value_counts().reset_index()

In [29]:
entity_counts

,index,dbpedia
0,"(DBpedia:Agent, DBpedia:Agent)",187352
1,"(DBpedia:FictionalCharacter, DBpedia:Agent)",120407
2,"(DBpedia:Agent, DBpedia:FictionalCharacter)",120407
3,"(DBpedia:FictionalCharacter, DBpedia:Fictional...",120407
4,"(DBpedia:Person, DBpedia:Agent)",41760
...,...,...
1636,"(DBpedia:Bird, DBpedia:Eukaryote)",1
1637,"(DBpedia:Medician, DBpedia:Agent)",1
1638,"(DBpedia:EurovisionSongContestEntry, DBpedia:M...",1
1639,"(DBpedia:EurovisionSongContestEntry, DBpedia:W...",1


In [30]:
unique_counts = entity_counts[entity_counts['index'].apply(lambda x: x[0] == x[1])]
unique_counts

,index,dbpedia
0,"(DBpedia:Agent, DBpedia:Agent)",187352
3,"(DBpedia:FictionalCharacter, DBpedia:Fictional...",120407
6,"(DBpedia:Person, DBpedia:Person)",41760
8,"(DBpedia:Location, DBpedia:Location)",30855
10,"(DBpedia:Place, DBpedia:Place)",30855
...,...,...
1585,"(DBpedia:SportFacility, DBpedia:SportFacility)",1
1609,"(DBpedia:CombinationDrug, DBpedia:CombinationD...",1
1622,"(DBpedia:Winery, DBpedia:Winery)",1
1629,"(DBpedia:TennisTournament, DBpedia:TennisTourn...",1


In [31]:
db_ents_with_id['dbpedia_top'] = db_ents_with_id['dbpedia'].apply(lambda x: x[0] if len(x) > 0 else '')
db_ents_with_id['dbpedia_top'].value_counts()

C:\Users\gokce\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


                                 138702
DBpedia:Agent                    137686
DBpedia:Person                    25565
DBpedia:Organisation              18888
DBpedia:Work                      15408
DBpedia:Region                    13498
DBpedia:PopulatedPlace            11589
DBpedia:Settlement                 3566
DBpedia:WrittenWork                2269
DBpedia:Disease                    2140
DBpedia:EthnicGroup                1987
DBpedia:FictionalCharacter         1973
DBpedia:TimeInterval               1749
DBpedia:Place                      1556
DBpedia:Language                   1508
DBpedia:Activity                   1466
DBpedia:SportsLeague                955
DBpedia:SportsTeam                  783
DBpedia:Food                        736
DBpedia:RacingDriver                671
DBpedia:SportsClub                  633
DBpedia:Stream                      564
DBpedia:TopicalConcept              473
DBpedia:Device                      413
DBpedia:AnatomicalStructure         398


In [32]:
db_ents_with_id_dbp['dbpedia'].value_counts()[:40]

DBpedia:Agent                     187352
DBpedia:FictionalCharacter        120407
DBpedia:Person                     41760
DBpedia:Location                   30855
DBpedia:Place                      30855
DBpedia:PopulatedPlace             28653
DBpedia:Organisation               25185
DBpedia:Work                       17677
DBpedia:AdministrativeRegion       13498
DBpedia:Region                     13498
DBpedia:Politician                 12542
DBpedia:Athlete                     8289
DBpedia:Country                     8144
DBpedia:Broadcaster                 8052
DBpedia:Settlement                  6597
DBpedia:BasketballPlayer            6247
DBpedia:TelevisionStation           5595
DBpedia:Website                     5374
DBpedia:MusicalWork                 4540
DBpedia:PoliticalParty              3711
DBpedia:Company                     2926
DBpedia:Song                        2407
DBpedia:BroadcastNetwork            2329
DBpedia:WrittenWork                 2273
DBpedia:Group   

In [33]:
db_entities[db_entities['ent_type'] == 'DBPEDIA_ENT'].iloc[39]['db_ents_list']

['Donald Trump Jr.',
 'DBPEDIA_ENT',
 'http://dbpedia.org/resource/Donald_Trump_Jr.',
 '1.0',
 'Http://xmlns.com/foaf/0.1/Person',
 'Wikidata:Q5',
 'Wikidata:Q24229398',
 'Wikidata:Q215627',
 'DUL:NaturalPerson',
 'DUL:Agent',
 'Schema:Person',
 'DBpedia:Agent',
 'DBpedia:Person']